Create surrogate neural population responses during natural behavior trials using variations on dynamical adapting neural models of Baker et al data.

In [1]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg
from disp import set_font_size
from my_stats import nanpearsonr

cc = np.concatenate

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS

Load behavioral data from Coen et al 2014

In [2]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
df_behav

,ID,FRAME,T,Q,S,P,F,FFV,FFA,FLS,FRS,FMFV,FMLS
0,0,0,0.000000,1,0,0,0,0.231,0.0000,0.159,0.077,-0.058526,0.274257
1,0,1,0.033300,1,0,0,0,0.314,0.0385,0.025,0.044,0.100623,0.298490
2,0,2,0.066600,1,0,0,0,0.308,0.0025,0.029,0.011,0.147629,0.271865
3,0,3,0.099900,1,0,0,0,0.319,0.0020,0.016,0.016,0.091174,0.306112
4,0,4,0.133200,1,0,0,0,0.312,-0.0315,0.030,0.002,0.065871,0.306439
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978565,275,27100,902.430902,1,0,0,0,0.217,0.0280,0.029,0.058,-0.171418,0.136184
4978566,275,27101,902.464202,1,0,0,0,0.210,-0.1045,0.138,0.322,0.032522,0.249171
4978567,275,27102,902.497502,1,0,0,0,0.008,-0.0910,0.034,0.260,0.033202,0.010844
4978568,275,27103,902.530803,1,0,0,0,0.028,0.0695,0.010,0.090,0.012059,0.027177


In [3]:
# split big df into dfs for individual trials
n_tr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(n_tr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# Simulation function

In [4]:
# define smln & obj function
def smlt(i_s, i_p, tau_r, tau_a, x_s, x_p):
    """Simulate response to song inputs."""
    t = np.arange(len(i_s))*DT
    r = np.nan*np.zeros(len(t))
    
    r[0] = 0
    a_s, a_p = 0, 0
    
    for ct, t_ in enumerate(t[1:], 1):
        a_s += ((DT/tau_a) * (-a_s + x_s*i_s[ct]))
        a_p += ((DT/tau_a) * (-a_p + x_p*i_p[ct]))
        r[ct] = r[ct-1] + (DT/tau_r) * (-r[ct-1] + (x_s - a_s)*i_s[ct] + (x_p - a_p)*i_p[ct])
    
    return r

# Baseline dynamical model params

In [5]:
# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,2.390705,0.604586


# No adaptation

In [6]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_nonadapt'
SAVE_PFX_DYN = 'mlv_baker_dyn_nonadapt'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_A'] = np.inf
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,inf,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,inf,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,inf,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,inf,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,inf,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,inf,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,inf,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,inf,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,inf,2.390705,0.604586


In [7]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [8]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Fixed adaptation timescale

In [6]:
TAU_A = 5
SAVE_DIR_DYN = f'data/simple/mlv/dyn_alt/baker_dyn_tau_a_{TAU_A}'
SAVE_PFX_DYN = f'mlv_baker_dyn_tau_a_{TAU_A}'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_A'] = TAU_A
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,5,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,5,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,5,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,5,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,5,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,5,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,5,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,5,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,5,2.390705,0.604586


In [7]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [8]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Stim-invariant adaptation

In [6]:
# define smln & obj function
def smlt_sia(i_s, i_p, tau_r, tau_a, x_s, x_p):
    """Simulate response to song inputs."""
    t = np.arange(len(i_s))*DT
    r = np.nan*np.zeros(len(t))
    
    r[0] = 0
    a = 0
    
    for ct, t_ in enumerate(t[1:], 1):
        a += ((DT/tau_a) * (-a + i_p[ct] + i_s[ct]))
        r[ct] = r[ct-1] + (DT/tau_r) * (-r[ct-1] + x_s*(1 - a)*i_s[ct] + x_p*(1 - a)*i_p[ct])
    
    return r

In [7]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_sia'
SAVE_PFX_DYN = 'mlv_baker_dyn_sia'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,2.390705,0.604586


In [8]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [9]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt_sia(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Max adaptation timescale

In [6]:
MAX_TAU_A = 1  # s

SAVE_DIR_DYN = f'data/simple/mlv/dyn_alt/baker_dyn_max_tau_a_{MAX_TAU_A}'
SAVE_PFX_DYN = f'mlv_baker_dyn_max_tau_a_{MAX_TAU_A}'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
tau_a = np.array(df_dyn_param['TAU_A'])
tau_a_fast = tau_a[tau_a <= MAX_TAU_A]
mslow_adapt = tau_a >= MAX_TAU_A
print(tau_a_fast)
df_dyn_param['TAU_A'][mslow_adapt] = np.random.choice(tau_a_fast, mslow_adapt.sum())
df_dyn_param

[0.64318249 0.37826713 0.38226739 0.4882192  0.49136376 0.63758011
 0.6429382  0.71577622 0.55544906 0.57454694 1.        ]


,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,0.555449,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,0.637580,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,0.382267,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,0.643182,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,0.555449,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,0.715776,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,0.555449,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,1.000000,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,0.491364,2.390705,0.604586


In [7]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [8]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Max adaptation timescale, fixed tau_r

In [7]:
MAX_TAU_A = 1  # s
TAU_R = 120  # s

SAVE_DIR_DYN = f'data/simple/mlv/dyn_alt/baker_dyn_max_tau_a_{MAX_TAU_A}_tau_r_{TAU_R}'
SAVE_PFX_DYN = f'mlv_baker_dyn_max_tau_a_{MAX_TAU_A}_tau_r_{TAU_R}'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
tau_a = np.array(df_dyn_param['TAU_A'])
tau_a_fast = tau_a[tau_a <= MAX_TAU_A]
mslow_adapt = tau_a >= MAX_TAU_A
print(tau_a_fast)
df_dyn_param['TAU_A'][mslow_adapt] = np.random.choice(tau_a_fast, mslow_adapt.sum())
df_dyn_param['TAU_R'] = TAU_R
df_dyn_param

[0.64318249 0.37826713 0.38226739 0.4882192  0.49136376 0.63758011
 0.6429382  0.71577622 0.55544906 0.57454694 1.        ]


,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,120,0.574547,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,120,0.382267,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,120,0.642938,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,120,0.488219,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,120,0.637580,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,120,0.643182,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,120,0.642938,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,120,0.574547,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,120,0.637580,2.390705,0.604586


In [8]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [17]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Max adaptation timescale, fixed tau_r, sine-selective only

In [9]:
MAX_TAU_A = 1  # s
TAU_R = 120  # s

SAVE_DIR_DYN = f'data/simple/mlv/dyn_alt/baker_dyn_max_tau_a_{MAX_TAU_A}_tau_r_{TAU_R}_sine'
SAVE_PFX_DYN = f'mlv_baker_dyn_max_tau_a_{MAX_TAU_A}_tau_r_{TAU_R}_sine'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
tau_a = np.array(df_dyn_param['TAU_A'])
tau_a_fast = tau_a[tau_a <= MAX_TAU_A]
mslow_adapt = tau_a >= MAX_TAU_A
print(tau_a_fast)
df_dyn_param['TAU_A'][mslow_adapt] = np.random.choice(tau_a_fast, mslow_adapt.sum())
df_dyn_param['TAU_R'] = TAU_R
df_dyn_param['X_P'] = 0
df_dyn_param

[0.64318249 0.37826713 0.38226739 0.4882192  0.49136376 0.63758011
 0.6429382  0.71577622 0.55544906 0.57454694 1.        ]


,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,120,0.491364,-4.958290,0
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,120,0.574547,-4.163967,0
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,120,1.000000,-6.248007,0
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,120,0.637580,-5.679735,0
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,120,0.637580,-4.415761,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,120,0.555449,-0.049828,0
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,120,0.378267,0.098451,0
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,120,0.378267,8.332263,0
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,120,0.574547,2.390705,0


In [10]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....

# Max adaptation timescale, fixed tau_r, pulse-selective only

In [6]:
MAX_TAU_A = 1  # s
TAU_R = 120  # s

SAVE_DIR_DYN = f'data/simple/mlv/dyn_alt/baker_dyn_max_tau_a_{MAX_TAU_A}_tau_r_{TAU_R}_pulse'
SAVE_PFX_DYN = f'mlv_baker_dyn_max_tau_a_{MAX_TAU_A}_tau_r_{TAU_R}_pulse'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
tau_a = np.array(df_dyn_param['TAU_A'])
tau_a_fast = tau_a[tau_a <= MAX_TAU_A]
mslow_adapt = tau_a >= MAX_TAU_A
print(tau_a_fast)
df_dyn_param['TAU_A'][mslow_adapt] = np.random.choice(tau_a_fast, mslow_adapt.sum())
df_dyn_param['TAU_R'] = TAU_R
df_dyn_param['X_S'] = 0
df_dyn_param

[0.64318249 0.37826713 0.38226739 0.4882192  0.49136376 0.63758011
 0.6429382  0.71577622 0.55544906 0.57454694 1.        ]


,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,120,0.637580,0,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,120,0.488219,0,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,120,0.637580,0,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,120,0.382267,0,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,120,0.378267,0,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,120,0.555449,0,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,120,0.555449,0,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,120,1.000000,0,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,120,0.574547,0,0.604586


In [7]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [8]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Fixed response timescale

In [6]:
TAU_R = 120

SAVE_DIR_DYN = f'data/simple/mlv/dyn_alt/baker_dyn_tau_r_{TAU_R}'
SAVE_PFX_DYN = f'mlv_baker_dyn_tau_r_{TAU_R}'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_R'] = TAU_R
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,120,30.336758,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,120,21.821295,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,120,15.058055,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,120,11.820576,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,120,55.496220,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,120,1440.994269,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,120,27.643238,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,120,6.155551,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,120,90.705238,2.390705,0.604586


In [7]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [8]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Fixed response timescale, no adaptation

In [14]:
TAU_R = 60

SAVE_DIR_DYN = f'data/simple/mlv/dyn_alt/baker_dyn_nonadapt_tau_r_{TAU_R}'
SAVE_PFX_DYN = f'mlv_baker_dyn_nonadapt_tau_r_{TAU_R}'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['TAU_R'] = TAU_R
df_dyn_param['TAU_A'] = np.inf
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,60,inf,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,60,inf,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,60,inf,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,60,inf,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,60,inf,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,60,inf,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,60,inf,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,60,inf,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,60,inf,2.390705,0.604586


In [15]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [16]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# No selectivity

In [7]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_non_select'
SAVE_PFX_DYN = 'mlv_baker_dyn_non_select'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
mn_selectivity = np.mean(np.array(df_dyn_param[['X_S', 'X_P']]), axis=1)
df_dyn_param['X_S'] = mn_selectivity
df_dyn_param['X_P'] = mn_selectivity
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,-3.777861,-3.777861
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,-2.818021,-2.818021
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,-4.171979,-4.171979
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,-3.898990,-3.898990
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,-3.639704,-3.639704
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,1.222414,1.222414
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,1.277917,1.277917
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,5.342740,5.342740
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,1.497645,1.497645


In [8]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [9]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Pure selectivity

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_pure_select'
SAVE_PFX_DYN = 'mlv_baker_dyn_pure_select'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
ms_selective = np.abs(df_dyn_param['X_S']) > np.abs(df_dyn_param['X_P'])
mp_selective = np.abs(df_dyn_param['X_P']) > np.abs(df_dyn_param['X_S'])
df_dyn_param['X_S'][mp_selective] = 0
df_dyn_param['X_P'][ms_selective] = 0

df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Depolarizing responses only

In [6]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_dep_only'
SAVE_PFX_DYN = 'mlv_baker_dyn_dep_only'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['X_S'][df_dyn_param['X_S'] < 0] = 0
df_dyn_param['X_P'][df_dyn_param['X_P'] < 0] = 0

df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,0.000000,0.000000
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,0.000000,0.000000
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,0.000000,0.000000
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,0.000000,0.000000
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,0.000000,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,2.390705,0.604586


In [7]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [8]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

# Hyperpolarizing responses only

In [6]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_hyp_only'
SAVE_PFX_DYN = 'mlv_baker_dyn_hyp_only'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['X_S'][df_dyn_param['X_S'] > 0] = 0
df_dyn_param['X_P'][df_dyn_param['X_P'] > 0] = 0

df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,0.000000,0.000000
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,0.000000,0.000000
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,0.000000,0.000000
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,0.000000,0.000000
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,0.000000,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,2.390705,0.604586


In [7]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

.

# Only sine selectivity

In [ ]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_sine_select'
SAVE_PFX_DYN = 'mlv_baker_dyn_sine_select'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['X_P'] = 0

df_dyn_param

In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

# Only pulse selectivity

In [6]:
SAVE_DIR_DYN = 'data/simple/mlv/dyn_alt/baker_dyn_pulse_select'
SAVE_PFX_DYN = 'mlv_baker_dyn_pulse_select'

if not os.path.exists(SAVE_DIR_DYN):
    os.makedirs(SAVE_DIR_DYN)

# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param['X_S'] = 0

df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,0,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,0,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,0,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,0,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,0,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,0,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,0,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,0,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,0,0.604586


In [ ]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [ ]:
# compute surrogate linear neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN, f'{SAVE_PFX_DYN}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))